Creating a python code to extract data from many tables in bigquery using a loop

Installing all necessary packages

In [1]:


#%pip install dbapi 
%pip install --upgrade google-cloud-bigquery
#%pip install db-dtypes



#import db_dtypes 
#import dbapi
#import datetime



Note: you may need to restart the kernel to use updated packages.


Creating a google cloud bigquery client using a service account

In [2]:
from google.cloud import bigquery # to run queries on google clouds bigquery
from google.oauth2 import service_account # to acess google cloud using a service account

credentials = service_account.Credentials.from_service_account_file(
    r"C:\Users\skicr\Downloads\alpine-tempo-392622-523114992507.json"
) 
google_cloud_project_id = 'alpine-tempo-392622' 
client = bigquery.Client(credentials=credentials,project=google_cloud_project_id)

In [ ]:
years = range(1950,2023)
queries = []
for year in years: 
    query=('''
            SELECT
                states.state,
                storm_data.deaths,
                storm_data.injuries,
                storm_data.damage
            FROM
              (
                SELECT
                  SUM(deaths_direct+deaths_indirect) AS deaths, 
                  SUM(injuries_direct+injuries_indirect)  AS injuries, 
                  SUM(damage_crops+damage_property) AS damage,
                  LPAD(state_fips_code,2,'0') as fips_code
                FROM
                  `bigquery-public-data.noaa_historic_severe_storms.storms_{year}`
                GROUP BY
                fips_code
              ) as storm_data
            RIGHT JOIN 
              `bigquery-public-data.geo_us_boundaries.states` AS states 
            ON 
              states.state_fips_code=storm_data.fips_code                               
            ORDER by 
              states.state
            ''').format(year=year)
    
    queries.append(query)


In [ ]:
%pip install pandas 

In [ ]:
import pandas as pd
results_list=[]

for x, query in enumerate(queries):
   query = client.query(queries[x])
   query_res = query.result().to_dataframe()
   query_res.insert(4,'year',years[x])
   results_list.append(query_res)
results_df=pd.concat(results_list,ignore_index=True)
results_df=results_df.convert_dtypes()
results_df.tail()

In [ ]:
%pip install cpi

In [ ]:
import cpi

inflation_factor = {}
for year in years:
   inflation_factor[year] = cpi.inflate(1,year)
inflation_factor

results_df['infl_adj_damage'] = results_df.apply(lambda row: row['damage']*inflation_factor[row['year']],axis=1)

results_df.dtypes



Scraping the US Censes Bureau Website to create columns for deaths and injuries normalized by population

In [ ]:

%pip install us 

In [ ]:
%env DC_STATEHOOD = 1
import us
def state_code_lookup(state_name):
    
    if type(state_name)==str:
        state_name=state_name.lstrip('.')
        if us.states.lookup(state_name) is not None:
            return us.states.lookup(state_name).abbr
    else:
        return "Unknown"

In [ ]:
import requests 
census_url = 'https://www.census.gov/data/tables/time-series/dec/popchange-data-text.html'
census_html = requests.get(census_url).text
popdf=pd.read_html(census_html)
popdf=popdf[0]
popdf.set_index('State or Region',inplace=True)
popdf.head()
consol_df = pd.DataFrame(columns=['state','population','year'])
cols = [*(popdf.columns)]
i=1
for col in cols:
    a=[*popdf[col]]
    for x,y in  enumerate(a):
        if a[x] == str(us.states.lookup(a[x])):
            if int(col[:4]) < 1950:
                break
            else:
                newrow=pd.DataFrame({'state':state_code_lookup(a[x]),'population':int(a[x+1]),'year':int(col[:4])},index=[0])
                consol_df=pd.concat([consol_df,newrow],ignore_index=True)


consol_df.head()



In [ ]:
%pip install numpy

In [ ]:
import numpy as np
consol_df = consol_df.astype({'population':np.int64})

Merging the two sets of data

In [ ]:

merged_df = results_df.merge(consol_df,on=['state','year'],how='left')
merged_df.head()



In [ ]:

merged_df.set_index(['state'],inplace=True)
states =[*merged_df.index.drop_duplicates()]

merged_df.sort_values(['state','year'],inplace=True)
for state in states:
    merged_df.loc[state,'population'].interpolate(method='linear',inplace=True)
merged_df.head(),merged_df.tail()


In [ ]:
merged_df.fillna(0,inplace=True)





In [ ]:
%pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

In [ ]:

url = 'https://www.census.gov/data/tables/time-series/demo/popest/2020s-state-total.html#v2022'
html=requests.get(url).text
soup = bs(html,'html.parser')
filetracks = soup.find_all('a',filetrack=True,href=True)
filetrack=filetracks[2].get('href')
print(filetrack)
   

In [ ]:
from urllib.parse import urlparse
from urllib.parse import urlunparse

In [ ]:
filetrack=urlunparse(urlparse(filetrack,scheme='https'))
filetrack


In [ ]:
%pip install openpyxl
import openpyxl


In [ ]:
more_pop_data = pd.read_excel(filetrack)
more_pop_data.head()


In [ ]:
trimmed_pop_data=more_pop_data.iloc[:,[0,3,4]]
trimmed_pop_data['State']=trimmed_pop_data.iloc[:,0].apply(state_code_lookup)
trimmed_pop_data.drop('table with row headers in column A and column headers in rows 3 through 5. (leading dots indicate sub-parts)',axis=1, inplace=True)
trimmed_pop_data.set_index('State', inplace=True)
trimmed_pop_data.rename(columns={'Unnamed: 3':'2021','Unnamed: 4':'2022'},inplace=True)
trimmed_pop_data.head(10)




In [ ]:
row_update=[]
for state in states:
    if state in [*(trimmed_pop_data.index)]:
        for year in [*(trimmed_pop_data.columns)]:
            row_update.append({'state':state,'year':year,'population':trimmed_pop_data.loc[state,year]})
yet_another_df=pd.DataFrame(row_update)
yet_another_df.set_index('state',inplace=True)
yet_another_df.head()


In [ ]:
for x in row_update:
    merged_df.loc[(x['state'],int(x['year'])),'population']=x['population']

In [ ]:

final_df=merged_df.reset_index()
final_df = final_df.astype({'population':np.int64,'infl_adj_damage':np.int64})
final_df['deaths/100k']=final_df.apply(lambda x: x['deaths']/x['population']*100000, axis=1)  
final_df['injuries/100k']=final_df.apply(lambda x: x['injuries']/x['population']*100000, axis=1)
final_df.head()

Some rows still don't have population data

In [ ]:
rows_to_drop=[*final_df.loc[final_df['population']==0].index]
final_df.drop(index=rows_to_drop,axis=0,inplace=True)

In [ ]:
final_df['deaths/100k']=final_df.apply(lambda x: x['deaths']/x['population']*100000, axis=1)  
final_df['injuries/100k']=final_df.apply(lambda x: x['injuries']/x['population']*100000, axis=1)
final_df.head()

In [ ]:
%pip install XlsxWriter


In [ ]:
import xlsxwriter
#with pd.ExcelWriter(r'c:\Users\skicr\Documents\Python Scripts\NOAA Storm Data\storm data by year.xlsx') as writer:
final_df.to_excel(pd.ExcelWriter(r'c:\Users\skicr\Documents\Python Scripts\NOAA Storm Data\storm data by year.xlsx'),index=False,)  
